In [1]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import EfficientNet
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm  # For CBAM
from timm.models.layers import SelectAdaptivePool2d
import numpy as np

c:\Users\Admin\anaconda3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# CBAM Module
class CBAM(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(CBAM, self).__init__()
        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // reduction_ratio, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction_ratio, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_att = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel Attention
        channel_weight = self.channel_att(x)
        x = x * channel_weight

        # Spatial Attention
        spatial_weight = torch.cat([torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]], dim=1)
        spatial_weight = self.spatial_att(spatial_weight)
        x = x * spatial_weight

        return x

In [3]:
# Custom Dataset
class CropDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = []
        self.image_paths = []

        # Collect classes
        for crop_name in os.listdir(root_dir):
            crop_path = os.path.join(root_dir, crop_name)
            if os.path.isdir(crop_path):
                for disease in os.listdir(crop_path):
                    disease_path = os.path.join(crop_path, disease)
                    if os.path.isdir(disease_path):
                        class_name = f"{crop_name}_{disease}"
                        self.classes.append(class_name)
                        for img in os.listdir(disease_path):
                            self.image_paths.append((os.path.join(disease_path, img), class_name))

        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, class_name = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image=np.array(image))['image']

        label = self.class_to_idx[class_name]
        return image, label


In [4]:
# Data Transformations
train_transform = A.Compose([
    A.Resize(224, 224),
    A.RandomCrop(200, 200),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [5]:
# Model with CBAM
class EfficientNet_CBAM(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNet_CBAM, self).__init__()
        self.model = EfficientNet.from_pretrained("efficientnet-b0", num_classes=num_classes)
        self.cbam = CBAM(in_channels=1280)  # Adjust based on EfficientNet output channels
        self.pool = SelectAdaptivePool2d(pool_type='avg', flatten=True)
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model.extract_features(x)
        x = self.cbam(x)
        x = self.pool(x)
        x = self.fc(x)
        return x

In [6]:
def train_efficientnet(data_dir, batch_size=64, epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    train_dir = os.path.join(data_dir, "train")
    test_dir = os.path.join(data_dir, "test")

    train_dataset = CropDiseaseDataset(train_dir, transform=train_transform)
    test_dataset = CropDiseaseDataset(test_dir, transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    num_classes = len(train_dataset.classes)
    model = EfficientNet_CBAM(num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_acc = 0.0
    scaler = torch.amp.GradScaler('cuda')  # Mixed Precision Training

    for epoch in range(epochs):  # ✅ Only one training loop
        model.train()  # ✅ Move model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        print(f"\nEpoch {epoch+1}/{epochs} - Training Start")
    
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.amp.autocast('cuda'):  # Use FP16 for faster training
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

          
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} Completed - Avg Loss: {avg_loss:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        test_acc = correct / total
        print(f"Validation Accuracy: {test_acc:.4f}")

        # Save best model
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ Model saved (Best Accuracy so far)!")

    print("🎉 Training complete.")

In [7]:
# Prediction Function
def predict(image_path, crop_name, model_path="best_model.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = EfficientNet_CBAM(num_classes=len(train_dataset.classes))
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Process Image
    image = Image.open(image_path).convert("RGB")
    image = test_transform(image=np.array(image))['image']
    image = image.unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        output = model(image)
        predicted_class = output.argmax(dim=1).item()

    return list(train_dataset.class_to_idx.keys())[predicted_class]


In [ ]:
# Train the Model
train_efficientnet("Processed_Data")

Using device: cuda
Loaded pretrained weights for efficientnet-b0

Epoch 1/10 - Training Start
started baby


In [1]:
#Clearing cache

# import torch
# import gc

# torch.cuda.empty_cache()  # Clears unused memory from the cache
# torch.cuda.ipc_collect()
# gc.collect()  # Force garbage collection
# torch.cuda.empty_cache()